# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head(10)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
Group_by = data.groupby(["CustomerID","ProductName"])[["Quantity"]].sum()
Group_by

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
Customer_ID = pd.pivot_table(data,values='Quantity', index=['ProductName'],
                    columns=['CustomerID'])
Customer_ID.fillna(0, inplace = True)
trans_customerID = Customer_ID.transpose()
trans_customerID

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,25.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [13]:
euclidean = pd.DataFrame(1/(1 + squareform(pdist(trans_customerID, 'euclidean'))),
                                index=Customer_ID.columns,
                                columns=Customer_ID.columns)

euclidean

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [14]:
similar_customer = euclidean[200].sort_values(ascending=False)[1:6]
similar_customer

CustomerID
1920    0.094414
1072    0.092610
3317    0.090094
3253    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [16]:
records_customers = data.loc[similar_customer.index]
records_customers

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,,
1920,20446,Cassandra,Weiss,533786,244,Beef - Rib Eye Aaa,6
1072,61288,Rosa,Andersen,2121381,17,"Wine - White, Mosel Gold",16
3317,38048,Rafael,Osborne,6732174,194,Lettuce - Frisee,10
3253,67682,Kimberly,Golden,2122683,194,Lettuce - Frisee,18
3909,55162,Felicia,Arnold,5942967,131,Anchovy Paste - 56 G Tube,14


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [17]:
qty_rank=records_customers.groupby(['ProductName'],as_index=False)['Quantity'].sum().sort_values(by='Quantity',ascending=False)
qty_rank

,ProductName,Quantity
2,Lettuce - Frisee,28
3,"Wine - White, Mosel Gold",16
0,Anchovy Paste - 56 G Tube,14
1,Beef - Rib Eye Aaa,6


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [18]:
df_merged = pd.merge(qty_rank, Customer_ID, on = "ProductName")
costumer_table = df_merged.sort_values(by= 'Quantity', ascending = False)[[200,'Quantity']].head()
costumer_table

,200,Quantity
0,0.0,28
1,0.0,16
2,0.0,14
3,0.0,6


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [19]:
product_recomentdations ={}

customer_list = data['CustomerID'].unique()

for customer in customer_list:
    customer=customer
    customer_recomend=euclidean[customer].sort_values(ascending=False)[1:6]
    total_quantity_example=data[data['CustomerID'].isin(customer_recomend.index)]
    total_quantity_by_product=total_quantity_example.groupby(['ProductName'],as_index=False).agg({'Quantity':'sum'})
    
    
    total_quantity_by_product=total_quantity_by_product.sort_values('Quantity',ascending=False)
    total_quantity_example_purchate=data[data["CustomerID"]==customer]
    all_products=pd.merge(total_quantity_by_product,total_quantity_example_purchate,how="left",on="ProductName")
    non_purchate_products=all_products[all_products['Quantity_y'].isnull()].sort_values('Quantity_x',
                                                                                        ascending=False)[0:5]
    non_purchate_products=non_purchate_products["ProductName"]
    product_recomentdations[customer]= list(non_purchate_products.values)

product_recomentdations

{61288: ['Mushrooms - Black, Dried',
  'Wine - Magnotta, Merlot Sr Vqa',
  'Chicken - Soup Base',
  'Wine - Chardonnay South',
  'Milk - 1%'],
 77352: ['Guinea Fowl',
  'Grenadine',
  'Ecolab - Mikroklene 4/4 L',
  'Oranges - Navel, 72',
  'Coconut - Shredded, Sweet'],
 40094: ['Water - Mineral, Natural',
  'Oregano - Dry, Rubbed',
  'Quiche Assorted',
  'Pasta - Orecchiette',
  'Beef - Inside Round'],
 23548: ['Wanton Wrap',
  'Banana Turning',
  'Flavouring - Orange',
  'Beer - Original Organic Lager',
  'Puree - Passion Fruit'],
 78981: ['Lettuce - Frisee',
  'Longos - Chicken Wings',
  'Pop Shoppe Cream Soda',
  'Sprouts - Alfalfa',
  'Beef - Inside Round'],
 83106: ['Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Soup - Campbells Tomato Ravioli',
  'Beef - Rib Eye Aaa',
  'Wine - Chablis 2003 Champs'],
 11253: ['Smirnoff Green Apple Twist',
  'Tomatoes Tear Drop',
  'Lettuce - Treviso',
  'Juice - Cranberry, 341 Ml',
  'Soup Campbells - Italian Wedding'],
 35107: ['Fla

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.